<a href="https://colab.research.google.com/github/ChoHadam/Insta_User_Profiling/blob/master/Insta_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests, re, json, urllib, datetime
import pandas as pd
from bs4 import BeautifulSoup

#search = '%EC%9D%B4%EC%8A%B9%EA%B8%B0'
#def insta(query):
#search = urllib.parse.quote(query)

# 헤더 저장
headers = {
            'cookie' : 'ig_did=1DB997A1-85F5-410B-A44B-DB3A31DE9BF2; mid=XxLtBgALAAHy9DlenU_IjVp8c-vI; csrftoken=w8fk3ZvDCg3GeMsgOGjj4c7C1J1CDDEv; ds_user_id=2164165712; sessionid=2164165712%3A0i2iNaTHNVLjRS%3A25; shbid=980; shbts=1597821443.3613222; rur=VLL; urlgen="{\"222.107.238.125\": 4766}:1k90Xa:Xh5aOhIn5p2sNhAMjIh4OesdfJk"',
            'user-agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36',
            }
            
# 해시 태그 설정
tags = "20대"

res = requests.get('https://www.instagram.com/explore/tags/{}/?__a=1'.format(tags), headers = headers)
res.encoding = 'utf-8'
js = json.loads(res.text)

In [3]:
user_id = []

for i in js['graphql']['hashtag']['edge_hashtag_to_media']['edges']:
    # user 이름으로 접속할 경우 게시글이 다른 게시물로 넘어감
    user_id.append(i['node']['owner']['id'])

user_id = list(set(user_id))
user_id[:5]

['8321534366', '1716835245', '40215342949', '39195862640', '624593782']

In [4]:
# 유저 게시글 저장

"""
post_user 형태
{"user_ID" : {"user_info" : "user_name", "post_content" : [{"post_code", "post_text", "post_location", "post_time", "post_like"},
                                                           {"post_code", "post_text", "post_location", "post_time", "post_like"}]}}}                                                
"""

import time

def insta_user_crawl(js, crawl_count):
    post_codes = []
    for i in range(len(js['data']['user']['edge_owner_to_timeline_media']['edges'])):
        crawl_count += 1
        if crawl_count > 500:
            time.sleep(300)
        user_id = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['id'] # 유저 아이디 기준으로 딕셔너리 생성
        post_code = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['shortcode']
        post_codes.append(post_code)
        new_post = {}
        if user_id not in new_post.keys() :
            new_post[user_id] = {}
        new_post[user_id]['user_name'] = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['owner']['username']
        if js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location'] != None:
            location = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['location']['name']
        else:
            location = None
        if 'post_content' not in new_post[user_id].keys():
            new_post[user_id]['post_content'] = []
        new_post[user_id]['post_content'].append({'post_code' : post_code,
                                                'post_text' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_caption']['edges'][0]['node']['text'], 
                                                'post_time' : (datetime.datetime.fromtimestamp(int(js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['taken_at_timestamp']))).strftime('%Y-%m-%d %H:%M:%S'),
                                                'post_location' : location,
                                                'post_like' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_preview_like']['count']})

        
        if user_id not in post_user.keys() :
            post_user[user_id] = new_post[user_id]
        else :
            post_user[user_id]['post_content'] = post_user[user_id]['post_content'] + new_post[user_id]['post_content']
        # 크롤링 진행 상황 출력
        if crawl_count % 100 == 0:
            print("크롤링 횟수 : " + crawl_count)

        return post_user, crawl_count, post_code

In [ ]:
post_user = {}
crawl_count = 0

for user in user_id:
    url = 'https://www.instagram.com/graphql/query/?query_hash=bfa387b2992c3a52dcbe447467b4b771&variables=%7B"id"%3A"{}"%2C"first"%3A12'.format(user)
    url_end = '%7D'
    res = requests.get(url + url_end, headers=headers)
    res.encoding = 'utf-8'
    js = json.loads(res.text)
    while True:
        if js['data']['user']['edge_owner_to_timeline_media']['page_info']['has_next_page'] == True: # 다음 페이지가 존재할 경우
            new_url = url + user + '%2C"after"%3A"{}"'.format(js['data']['user']['edge_owner_to_timeline_media']['page_info']['end_cursor'])+ url_end
            print(new_url)
            res1 = requests.get(new_url)
            res1.encoding='utf-8'
            js = json.loads(res1.text)
            insta_user_crawl(js, crawl_count)
        else:
            print(new_url)
            res1 = requests.get(new_url)
            res1.encoding='utf-8'
            js = json.loads(res1.text)
            insta_user_crawl(js, crawl_count)
            break

post_user

In [16]:
post_user['8321534366']

{'post_content': [{'post_code': 'CCfIDSdpuc1',
   'post_like': 32,
   'post_location': None,
   'post_text': '오늘 웨스턴빌리프 첫타임신랑신부님\n.\n.\n.\n이렇게 예쁘실수가..\n너무 예쁘세요^^\n\n두분덕분에 웨딩홀이 더 반짝반짝\n.\n.\n.\n코로나로 인해 예식연기..\n그간 마음고생 많으셨죠..?\n.\n.\n오늘 너무 이쁘시네요^^\n\n항상 행복하네요 ❤❤\n\n더욱 화사해진 웨스턴 빌리프 웨딩홀에 여러분들을 초대합니다☝\n⠀\n\n━━━━━━━━━💍𝐖𝐞𝐬𝐭𝐞𝐫𝐧 𝐁𝐞𝐥𝐢𝐞𝐟━──··\n⠀\n⠀#일산웨딩홀 ✦#웨스턴빌리프 웨딩홀✦\n⠀\n┊ 웨딩 ┊ 행사 ┊ 전시 ┊ 채플웨딩 ┊ 문화공간 ┊\n⠀\n✲문의✲\nᴛᴇʟ 031-815-8588\n⠀\n잊을 수 없는 소중한 추억을 선물합니다🤵👰💕\n━━━━━━━━━━━━━━━━━━──··\n✓ 일산 유일의 단독 채플 예식\n✓ 2시간 동안의 여유로운 진행\n✓ one-stop 시스템으로 편안한 동선.\n✓ 완벽한 주차시설, 지하철역 수시 셔틀버스 운행\n✓ 빠르고 편한 상담을 위해 방문 전 사전예약제 진행\n✓ 온라인 예약 가능\n.\n.\n#일산 #파주 #일산웨딩홀추천 #파주웨딩홀 #좋아요\n#스몰웨딩장소 #스몰웨딩홀 #예식장추천 #결혼준비\n#럽스타그램 #웨딩홀추천 #웨딩홀투어 #웨딩홀문의\n#웨딩홀할인 #웨딩박람회 #단독웨딩홀 #채플웨딩\n#채플웨딩홀 #예비신랑 #예비신부 #선팔하면맞팔\n#맞팔100 #좋아요반사 #일산웨딩홀 #김포웨딩홀 #블랑해리 #레이나웨딩',
   'post_time': '2020-07-11 03:15:04'},
  {'post_code': 'B-EsBOiJicE',
   'post_like': 17,
   'post_location': None,
   'post_text': '-\n#세스코 전문살균 6차 실시 👍🏻\n방문하시는 모든 귀빈 여러분들의 걱정을 덜어드리기 위해서 전문업체에

In [129]:
#post_reply 형태 = {"post_code" : {"user_code", "reply_user_name", "reply_content" : [{"reply_text", "reply_time", "reply_like"}]}}
post_reply = {}

for i in range(len(js['data']['user']['edge_owner_to_timeline_media']['edges'])):
    new_reply = {}
    post_code = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['shortcode']
    if js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['count'] != 0:
        if post_code not in new_reply.keys() :
            new_reply[post_code] = {}
        if 'reply_content' not in new_reply[post_code].keys():
            new_reply[post_code]['reply_content'] = []
        if js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][i]['node']["edge_liked_by"] == None:
            like = 0
        else:
            like = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][i]['node']["edge_liked_by"]['count']


        for i in range(len(js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'])):
            new_reply[post_code]['reply_user_name'] = js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][i]['node']['owner']['username']
            new_reply[post_code]['reply_content'].append({'reply_text' : js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][i]['node']['text'],
                                                          'reply_time' : (datetime.datetime.fromtimestamp(int(js['data']['user']['edge_owner_to_timeline_media']['edges'][i]['node']['edge_media_to_comment']['edges'][i]['node']['created_at']))).strftime('%Y-%m-%d %H:%M:%S'),
                                                          'reply_like' : like})                
        if post_code not in post_reply.keys() :
            post_reply[post_code] = new_reply[post_code]
        else :
            post_reply[post_code]['reply_content'] = post_reply[post_code]['reply_content'] + new_reply[post_code]['reply_content']

post_reply
    # for 
    #     new_reply[post_code]['reply_user_name'] =

KeyError: ignored